# Data Cleaning 

In [2]:
# necessary to import db_connector script
import sys
import os

# Get the absolute path of the project root
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))

# Add project root to sys.path
if project_root not in sys.path:
    sys.path.insert(0, project_root)

In [3]:
import pandas as pd
import numpy as np 
import datetime 
import re 

from db_connector import load_from_excel

In [4]:
data = load_from_excel()

# Make copies to avoid modifying originals
cleaned_data = {key: df.copy() for key, df in data.items()}

print("Data loaded successfully.")

Data loaded successfully.


## Clean Requisitions Table

In [ ]:
print("\n=== Cleaning Requisitions Table ===")

# 1. Handling missing values 
# a. Fix RECRUITER field (30.6% missing)

print("\n1. Handling missing values:")
missing_recruiter = cleaned_data['requisitions']['RECRUITER'].isnull().sum()
print(f"   - Missing RECRUITER values: {missing_recruiter} ({missing_recruiter/len(cleaned_data['requisitions'])*100:.2f}%)")

# Use RECRUITER_ID to fill in missing RECRUITER values where possible
recruiter_map = cleaned_data['requisitions'][cleaned_data['requisitions']['RECRUITER'].notna()].groupby('RECRUITER_ID')['RECRUITER'].first().to_dict()
print(f"   - Found {len(recruiter_map)} unique RECRUITER_ID to RECRUITER mappings")

# Fill missing values using the mapping
before_fill = cleaned_data['requisitions']['RECRUITER'].isnull().sum()
cleaned_data['requisitions']['RECRUITER'] = cleaned_data['requisitions'].apply(
    lambda row: recruiter_map.get(row['RECRUITER_ID']) if pd.isnull(row['RECRUITER']) and row['RECRUITER_ID'] in recruiter_map else row['RECRUITER'], 
    axis=1
)
after_fill = cleaned_data['requisitions']['RECRUITER'].isnull().sum()
print(f"   - Filled {before_fill - after_fill} missing RECRUITER values")
print(f"   - Remaining missing RECRUITER values: {after_fill} ({after_fill/len(cleaned_data['requisitions'])*100:.2f}%)")

# b. Add flag for missing CLOSE_DATE (5.9% missing) - these are likely still open
cleaned_data['requisitions']['MISSING_CLOSE_DATE'] = cleaned_data['requisitions']['CLOSE_DATE'].isnull()
print(f"   - Flagged {cleaned_data['requisitions']['MISSING_CLOSE_DATE'].sum()} records with missing CLOSE_DATE")

# 2. Check for unusual NUMBER_OF_OPENINGS
high_openings = cleaned_data['requisitions'][cleaned_data['requisitions']['NUMBER_OF_OPENINGS'] > 10]
print(f"\n2. Found {len(high_openings)} requisitions with more than 10 openings")

# 3. Create a cleaned version ready for analysis
cleaned_requisitions = cleaned_data['requisitions'].copy()
print("\nRequisitions cleaning completed.")
print(f"Original shape: {data['requisitions'].shape}, Cleaned shape: {cleaned_requisitions.shape}")


=== Cleaning Requisitions Dataset ===

2. Handling missing values:
   - Missing RECRUITER values: 0 (0.00%)
   - Found 37 unique RECRUITER_ID to RECRUITER mappings
   - Filled 0 missing RECRUITER values
   - Remaining missing RECRUITER values: 0 (0.00%)
   - Flagged 296 records with missing CLOSE_DATE

3. Found 25 requisitions with more than 10 openings

Requisitions cleaning completed.
Original shape: (5025, 11), Cleaned shape: (5025, 12)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5025 entries, 0 to 5024
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   REQUISITION_ID      5025 non-null   int64         
 1   REQUISITION_UID     5025 non-null   int64         
 2   STATUS_IN           5025 non-null   object        
 3   OPEN_DATE           5025 non-null   datetime64[ns]
 4   CLOSE_DATE          4729 non-null   datetime64[ns]
 5   NUMBER_OF_OPENINGS  5025 non-null   int64         


## Clean Candidate Dataset

In [12]:
print("\n=== Cleaning Candidate Dataset ===")

# 1. Handle duplicates
duplicate_count = cleaned_data['candidate'].duplicated().sum()
print(f"1. Found {duplicate_count} duplicate records ({duplicate_count/len(cleaned_data['candidate'])*100:.2f}%)")
cleaned_data['candidate'] = cleaned_data['candidate'].drop_duplicates()
print(f"   - Removed {duplicate_count} duplicate records")

# 2. Fix date columns
date_cols = [col for col in cleaned_data['candidate'].columns if 'DATE' in col]
for col in date_cols:
    # Convert to datetime, set errors to coerce to handle invalid dates
    if cleaned_data['candidate'][col].dtype != 'datetime64[ns]':
        cleaned_data['candidate'][col] = pd.to_datetime(cleaned_data['candidate'][col], errors='coerce')
        print(f"   - Converted {col} to datetime format")

# 3. Handle records with illogical date sequences
# Check submission_date to historical_status_start_date
illogical_dates = cleaned_data['candidate'][
    (cleaned_data['candidate']['SUBMISSION_DATE'].notna()) & 
    (cleaned_data['candidate']['HISTORICAL_STATUS_START_DATE'].notna()) & 
    (cleaned_data['candidate']['HISTORICAL_STATUS_START_DATE'] < cleaned_data['candidate']['SUBMISSION_DATE'])
]
print(f"\n3. Found {len(illogical_dates)} records where status start date is before submission date")
print("   - These will be flagged but kept in the dataset")
cleaned_data['candidate']['ILLOGICAL_DATE_FLAG'] = False
cleaned_data['candidate'].loc[illogical_dates.index, 'ILLOGICAL_DATE_FLAG'] = True

# 4. Handle missing CANDIDATE_ID values
missing_ids = cleaned_data['candidate'][cleaned_data['candidate']['CANDIDATE_ID'].isna()]
print(f"\n4. Found {len(missing_ids)} records with missing CANDIDATE_ID")
print("   - These will be flagged but kept in the dataset")
cleaned_data['candidate']['MISSING_ID_FLAG'] = cleaned_data['candidate']['CANDIDATE_ID'].isna()

# 5. Create a clean version ready for analysis
cleaned_candidate = cleaned_data['candidate'].copy()
print("\nCandidate cleaning completed.")
print(f"Original shape: {data['candidate'].shape}, Cleaned shape: {cleaned_candidate.shape}")



=== Cleaning Candidate Dataset ===
1. Found 0 duplicate records (0.00%)
   - Removed 0 duplicate records
   - Converted SUBMISSION_DATE to datetime format
   - Converted CANDIDATE_ID to datetime format
   - Converted CANDIDATE_HISTORICAL_STATUS to datetime format

3. Found 5723 records where status start date is before submission date
   - These will be flagged but kept in the dataset

4. Found 17 records with missing CANDIDATE_ID
   - These will be flagged but kept in the dataset

Candidate cleaning completed.
Original shape: (615707, 9), Cleaned shape: (615707, 11)


/var/folders/3s/27y6xzfx5j55fg4s9swmhjsc0000gn/T/ipykernel_28993/4223145300.py:14: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  cleaned_data['candidate'][col] = pd.to_datetime(cleaned_data['candidate'][col], errors='coerce')


## Clean Candidate Status Table

In [10]:
# 1. Remove duplicates
duplicate_count = cleaned_data['candidate_status'].duplicated().sum()
print(f"\n1. Found {duplicate_count} duplicate records")
cleaned_data['candidate_status'] = cleaned_data['candidate_status'].drop_duplicates()
print(f"   - Removed {duplicate_count} duplicate records")

# 3. Create a clean version ready for analysis
cleaned_candidate_status = cleaned_data['candidate_status'].copy()
print("\nCandidate Status cleaning completed.")
print(f"Original shape: {data['candidate_status'].shape}, Cleaned shape: {cleaned_candidate_status.shape}")


1. Found 0 duplicate records
   - Removed 0 duplicate records

Candidate Status cleaning completed.
Original shape: (16, 2), Cleaned shape: (16, 2)


In [13]:
# Save cleaned data to cleaned_data dictionary
cleaned_data = {
    'requisitions': cleaned_requisitions,
    'candidate': cleaned_candidate,
    'candidate_status': cleaned_candidate_status,
    'department': cleaned_data['department']
}

print("\n=== Summary of Cleaning Results ===")
for table_name, df in cleaned_data.items():
    original_shape = data[table_name].shape
    cleaned_shape = df.shape
    print(f"{table_name}: Original {original_shape} -> Cleaned {cleaned_shape}")

print("\nCleaning process completed successfully!")


=== Summary of Cleaning Results ===
requisitions: Original (5025, 11) -> Cleaned (5025, 12)
candidate: Original (615707, 9) -> Cleaned (615707, 11)
candidate_status: Original (16, 2) -> Cleaned (16, 2)
department: Original (392, 4) -> Cleaned (392, 4)

Cleaning process completed successfully!
